In [1]:
from dotenv import load_dotenv
load_dotenv()

False

In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

model = ChatOpenAI(
    model="gpt-4o-mini",   # usa un modelo que tengas habilitado
    temperature=0
)
print("🤖 Modelo listo")

🤖 Modelo listo


In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
import os

In [4]:
import os, json
from datetime import datetime

# === Función para auditar código ===
def audit_code(file_path: str):
    """
    Realiza una auditoría de seguridad del archivo indicado
    usando el modelo inicializado en LangChain.
    """
    with open(file_path, "r", encoding="utf-8", errors="ignore") as f:
        code = f.read()

    prompt_template = """Eres un experto en seguridad OWASP.
    Analiza el siguiente código en busca de vulnerabilidades y responde en JSON válido.
    El formato debe ser:
    {{
      "file": "{file_name}",
      "vulnerabilities": [
        {{
          "title": "...",
          "severity": "CRITICAL|HIGH|MEDIUM|LOW",
          "cwe_id": "CWE-###",
          "description": "...",
          "line_numbers": [<int>],
          "recommendation": "..."
        }}
      ]
    }}

    Código:
    ```{code_lang}
    {code}
    ```"""

    # Crea el prompt
    prompt = prompt_template.format(
        file_name=os.path.basename(file_path),
        code_lang="C# (ASP.NET WebForms)",
        code=code[:4000]  # truncar si el archivo es muy largo
    )

    response = model.invoke(prompt)

    # Guarda el resultado en carpeta results
    os.makedirs("results", exist_ok=True)
    ts = datetime.now().strftime("%Y%m%d_%H%M%S")
    out_file = f"results/{os.path.basename(file_path)}_audit_{ts}.json"

    try:
        data = json.loads(response.content)
    except Exception:
        data = {"error": "Respuesta no es JSON válido", "raw": response.content}

    with open(out_file, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

    print(f"✅ Auditoría guardada en {out_file}")
    return data


In [5]:
import glob

# Buscar archivos .cs, .aspx, .asax dentro de /codigos
files = []
for ext in ("*.cs", "*.aspx", "*.asax", "*.config"):
    files.extend(glob.glob(os.path.join("codigos", "**", ext), recursive=True))

print(f"Encontrados {len(files)} archivos para auditar")

# Auditar todos
for f in files:
    print(f"🔍 Auditando: {f}")
    audit_code(f)


Encontrados 0 archivos para auditar


In [6]:
import json, glob

summary = {"CRITICAL":0, "HIGH":0, "MEDIUM":0, "LOW":0, "UNKNOWN":0}

for file in glob.glob("results/*_audit_*.json"):
    with open(file, "r", encoding="utf-8") as f:
        data = json.load(f)
        for v in data.get("vulnerabilities", []):
            sev = v.get("severity", "UNKNOWN").upper()
            if sev in summary: summary[sev]+=1
            else: summary["UNKNOWN"]+=1

print("📊 Resumen general:")
for s, n in summary.items():
    print(f"{s:9}: {n}")


📊 Resumen general:
CRITICAL : 0
HIGH     : 0
MEDIUM   : 0
LOW      : 0
UNKNOWN  : 0


In [7]:
# === Auditoría de carpeta de código y guardado de resultados ===
import os, json
from pathlib import Path
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt

# Rutas del repo
REPO_DIR     = Path(".").resolve()
CODE_DIR     = REPO_DIR / "codigos"              # Carpeta principal de código
CODE_INSEG   = REPO_DIR / "codigos_inseguro"     # (Opcional) versión insegura
CODE_SEG     = REPO_DIR / "codigos_seguro"       # (Opcional) versión segura
RESULTS_DIR  = REPO_DIR / "results"              # Carpeta donde se guardan los JSON
RESULTS_DIR.mkdir(exist_ok=True)

# Extensiones a auditar
EXTS = {".cs", ".aspx", ".config", ".cshtml", ".vb", ".xml"}

def archivos_en(carpeta: Path):
    if not carpeta.exists():
        return []
    return [p for p in carpeta.rglob("*") if p.suffix.lower() in EXTS and p.is_file()]

def auditar_carpeta(carpeta_codigo: Path, carpeta_resultados: Path, etiqueta: str):
    """
    Recorre la carpeta de código, llama a tu función de auditoría por archivo y
    guarda un JSON por cada archivo analizado en 'carpeta_resultados'.
    El nombre del archivo resultante incluye la etiqueta y un timestamp.
    """
    print(f"▶ Auditando: {carpeta_codigo} -> {carpeta_resultados} ({etiqueta})")
    carpeta_resultados.mkdir(exist_ok=True, parents=True)
    ts = datetime.now().strftime("%Y%m%d_%H%M%S")

    total = 0
    for file_path in archivos_en(carpeta_codigo):
        total += 1
        # === LLAMADA A TU FUNCIÓN YA DEFINIDA ARRIBA ===
        # Debe retornar un dict con el análisis o bien guardar internamente el JSON.
        # Si tu función ya guarda el JSON, puedes omitir el bloque "with open(...)"
        # y sólo usar la llamada. Aquí asumimos que retorna un dict 'resultado'.
        try:
            resultado = audit_file_with_ai(str(file_path))  # <-- usa tu función anterior
        except NameError:
            raise RuntimeError(
                "No se encontró la función 'audit_file_with_ai'. "
                "Define arriba tu función que audite un archivo y devuelva un dict."
            )

        # Guardar cada resultado como JSON individual
        out_name = f"{etiqueta}_{file_path.stem}_audit_{ts}.json"
        out_path = carpeta_resultados / out_name
        with open(out_path, "w", encoding="utf-8") as f:
            json.dump(resultado, f, ensure_ascii=False, indent=2)
    print(f"✔ Archivos auditados: {total}")

def normaliza_severidad(s: str) -> str:
    if not s: return "low"
    s = s.strip().lower()
    if s.startswith("crit"): return "critical"
    if s.startswith("alt"):  return "high"
    if s.startswith("med"):  return "medium"
    if s.startswith("baj"):  return "low"
    return s if s in {"critical","high","medium","low"} else "low"

def resume_resultados(carpeta_resultados: Path, filtro_etiqueta: str = None) -> dict:
    """
    Lee todos los JSON en 'carpeta_resultados' (opcionalmente filtrando por etiqueta en el nombre)
    y cuenta vulnerabilidades por severidad. Se espera que cada JSON tenga una lista de
    vulnerabilidades con campo 'severity' o similar.
    """
    counts = {"critical": 0, "high": 0, "medium": 0, "low": 0}
    total_json = 0

    for p in carpeta_resultados.glob("*.json"):
        if filtro_etiqueta and not p.name.startswith(filtro_etiqueta + "_"):
            continue
        total_json += 1
        try:
            data = json.loads(p.read_text(encoding="utf-8"))
        except Exception:
            continue

        # Adapta a tu estructura: intentamos varias claves comunes
        vulns = []
        if isinstance(data, dict):
            if "vulnerabilities" in data and isinstance(data["vulnerabilities"], list):
                vulns = data["vulnerabilities"]
            elif "issues" in data and isinstance(data["issues"], list):
                vulns = data["issues"]

        for v in vulns:
            sev = None
            if isinstance(v, dict):
                sev = v.get("severity") or v.get("level") or v.get("severity_level")
            counts[normaliza_severidad(str(sev))] += 1

    total_vulns = sum(counts.values())
    # Estructura resumen (compat con tu snippet original)
    resumen = {
        "critical": counts["critical"],
        "high": counts["high"],
        "medium": counts["medium"],
        "low": counts["low"],
        "vulnerabilities": total_vulns,
        "archivos_analizados": total_json,
    }
    return resumen

# === 1) Auditar carpeta principal 'codigos/' ===
if CODE_DIR.exists():
    auditar_carpeta(CODE_DIR, RESULTS_DIR, etiqueta="codigos")

# === 2) (Opcional) Auditar dos versiones para comparación ===
security_results = None
secure_results   = None

if CODE_INSEG.exists():
    auditar_carpeta(CODE_INSEG, RESULTS_DIR, etiqueta="inseguro")
    security_results = resume_resultados(RESULTS_DIR, filtro_etiqueta="inseguro")

if CODE_SEG.exists():
    auditar_carpeta(CODE_SEG, RESULTS_DIR, etiqueta="seguro")
    secure_results = resume_resultados(RESULTS_DIR, filtro_etiqueta="seguro")

# Si no hay versiones separadas, hacemos un resumen único de 'codigos/'
if security_results is None and secure_results is None:
    unico = resume_resultados(RESULTS_DIR, filtro_etiqueta="codigos")
    print("Resumen (carpeta 'codigos/'):")
    print(unico)

# === 3) Construir la comparación (si hay dos versiones) y graficar ===
if security_results and secure_results:
    comparison_data = pd.DataFrame([
        {
            'Version': 'Código Inseguro',
            'High':   security_results.get('high', 0),
            'Medium': security_results.get('medium', 0),
            'Low':    security_results.get('low', 0)
        },
        {
            'Version': 'Código Seguro',
            'High':   secure_results.get('high', 0),
            'Medium': secure_results.get('medium', 0),
            'Low':    secure_results.get('low', 0)
        }
    ])

    fig, ax = plt.subplots(figsize=(10, 6))
    # Nota: no fijamos colores para mantenerlo genérico
    comparison_data.set_index('Version')[['High', 'Medium', 'Low']].plot(kind='bar', ax=ax)
    plt.title('Comparación de Vulnerabilidades: Código Inseguro vs Seguro', fontsize=14, fontweight='bold')
    plt.xlabel('Versión del Código', fontsize=12)
    plt.ylabel('Número de Vulnerabilidades', fontsize=12)
    plt.legend(title='Severidad', loc='upper right')
    plt.xticks(rotation=0)
    plt.tight_layout()
    plt.show()

    reduccion = security_results.get('vulnerabilities', 0) - secure_results.get('vulnerabilities', 0)
    print("\n📊 Conclusión:")
    print(f"Reducción de vulnerabilidades: {reduccion}")

else:
    print("\nℹ No se detectaron ambas versiones ('codigos_inseguro/' y 'codigos_seguro/').")
    print("   Se generó/actualizó el resumen único de la carpeta 'codigos/' y los JSON en 'results/'.")


ModuleNotFoundError: No module named 'pandas'